In [1]:
import duckdb
from lonboard import HeatmapLayer, Map
from lonboard.basemap import CartoStyle, MaplibreBasemap

In [2]:
con = duckdb.connect()
duckdb.install_extension("spatial", connection=con)
duckdb.load_extension("spatial", connection=con)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
sql = """
CREATE TABLE rides AS
SELECT
    *,
    ST_Point(dropoff_longitude, dropoff_latitude) as geometry
FROM "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-03.parquet"
WHERE
    dropoff_longitude >= -74.6087 AND
    dropoff_latitude >= 40.2738 AND
    dropoff_longitude <= -73.4928 AND
    dropoff_latitude <= 41.1757 AND
    total_amount > 0
LIMIT 1_000_000;
"""
con.execute(sql)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
con.table("rides")

┌───────────┬─────────────────────┬─────────────────────┬─────────────────┬────────────────────┬────────────────────┬─────────────────┬───────────┬────────────────────┬────────────────────┬──────────────────┬──────────────┬───────────────────┬───────────┬─────────┬────────────┬──────────────┬───────────────────┬───────────────────┬──────────────────────────────────────┐
│ vendor_id │   pickup_datetime   │  dropoff_datetime   │ passenger_count │   trip_distance    │  pickup_longitude  │ pickup_latitude │ rate_code │ store_and_fwd_flag │ dropoff_longitude  │ dropoff_latitude │ payment_type │    fare_amount    │ surcharge │ mta_tax │ tip_amount │ tolls_amount │   total_amount    │ __index_level_0__ │               geometry               │
│  varchar  │       varchar       │       varchar       │      int64      │       double       │       double       │     double      │   int64   │      varchar       │       double       │      double      │   varchar    │      double       │  double   

In [30]:
layer = HeatmapLayer.from_duckdb(con.table("rides"), con)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [36]:
bmap = MaplibreBasemap(mode='reverse-controlled', style = CartoStyle.DarkMatter)
# basemap = bmap
m = Map(layer, basemap = bmap, height=700, view_state={"longitude": -73.9790671, "latitude": 40.7594051, "zoom": 11})
m